# Problem Set 4
## Fundamentals of Simulation Methods, WiSe 20/21

#### Author:    Elias Olofsson

#### Version information:
        2020-12-02: v.1.0. First public release. 


In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Exercise 1 - The 2-body problem: Orbit of planet around the Sun (8 pts)